In [1]:
sc

Intitializing Scala interpreter ...

Spark Web UI available at http://sparkly:4040
SparkContext available as 'sc' (version = 2.4.0, master = local[*], app id = local-1553191813702)
SparkSession available as 'spark'


res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@726635c5


In [47]:
val Program=spark.read.option("header","true").parquet("hdfs://172.18.0.2:8020/Project/Data/Program/program.parquet")
Program.printSchema()

Program.createOrReplaceTempView("Program_view")
val Program_df = spark.sql("""select *
                            from Program_view where EVENT_DATE > '2016-12-31' 
                            """)
// Program_df.show(20)


root
 |-- PROGRAM_ID: string (nullable = true)
 |-- SUBCATEGORY_ID: string (nullable = true)
 |-- PROGRAM_SOURCE_ID: string (nullable = true)
 |-- PROGRAM_AIRING_TYPE_ID: string (nullable = true)
 |-- CATEGORY_ID: string (nullable = true)
 |-- CAPTION_ID: string (nullable = true)
 |-- PROGRAM_COLOR_TYPE_ID: string (nullable = true)
 |-- MASTER_TITLE: string (nullable = true)
 |-- RELEASE_YEAR: string (nullable = true)
 |-- EPISODE_TITLE: string (nullable = true)
 |-- EPISODE_NUMBER: string (nullable = true)
 |-- STEREO_ENABLED_YN: string (nullable = true)
 |-- STAR_RATING: string (nullable = true)
 |-- SERIES_YN: string (nullable = true)
 |-- RATING_ID_TV_US: string (nullable = true)
 |-- RATING_ID_MOVIE_US: string (nullable = true)
 |-- SERIES_MASTER_YN: string (nullable = true)
 |-- PARENT_PROGRAM_ID: string (nullable = true)
 |-- RUNTIME: string (nullable = true)
 |-- HDTV_YN: string (nullable = true)
 |-- BRACKET_TEXT: string (nullable = true)
 |-- PART_NUMBER: string (nullable = t

Program: org.apache.spark.sql.DataFrame = [PROGRAM_ID: string, SUBCATEGORY_ID: string ... 29 more fields]
Program_df: org.apache.spark.sql.DataFrame = [PROGRAM_ID: string, SUBCATEGORY_ID: string ... 29 more fields]


In [67]:
spark.sql("""
select distinct PROGRAM_ID,MASTER_TITLE,PARENT_PROGRAM_ID, RUNTIME/3600 as RT_HRS from Program_view 
order by 4 desc
""")
.show()

+----------+--------------------+-----------------+-----------------+
|PROGRAM_ID|        MASTER_TITLE|PARENT_PROGRAM_ID|           RT_HRS|
+----------+--------------------+-----------------+-----------------+
|  28118023|             Pirates|                N|7829.239444444444|
|  28144138|Lezioni dall'audi...|                N|7821.910277777778|
|  28206174|Lezioni dall'audi...|                N|7821.910277777778|
|  28158160|Equitazione: Bree...|                N|7821.709444444445|
|  28135499|Perfect Storms - ...|                N|7821.678055555555|
|  28157960|Snooker: Torneo i...|                N|7821.653888888889|
|  28158050|Snooker: Torneo i...|                N|7821.653888888889|
|  28158179|Snooker: Torneo i...|                N|7821.653888888889|
|  28157991|Snooker: Torneo i...|                N|7821.653888888889|
|  28158068|Snooker: Torneo i...|                N|7821.653888888889|
|  28158004|Snooker: Torneo i...|                N|7821.653888888889|
|  28210016|        

In [89]:

spark.sql("""

select A.*,B.CT, A.CT*100/B.CT as PCT
from
(select PARENT_PROGRAM_ID,count(distinct PROGRAM_ID) as CT  from Program_view 
where 
EVENT_DATE > '2017-12-31' and 
EVENT_DATE < '2019-01-01' 
group by PARENT_PROGRAM_ID) A
cross join
(select sum(1) as CT  from Program_view where 
EVENT_DATE > '2017-12-31' and 
EVENT_DATE < '2019-01-01' ) B

order by 2 desc
""")
.show()

+-----------------+----+-----+-------------------+
|PARENT_PROGRAM_ID|  CT|   CT|                PCT|
+-----------------+----+-----+-------------------+
|             null|9247|26099|  35.43047626345837|
|          1055967| 172|26099| 0.6590290815740066|
|          5665213|  73|26099|0.27970420322617723|
|          8871523|  69|26099| 0.2643779455151538|
|              118|  58|26099|0.22223073680983946|
|          1471955|  55|26099| 0.2107360435265719|
|         10367492|  54|26099|0.20690447909881604|
|          5121861|  53|26099| 0.2030729146710602|
|         16387971|  52|26099|0.19924135024330433|
|          1568565|  51|26099| 0.1954097858155485|
|          4373338|  47|26099|0.18008352810452508|
|          4227396|  46|26099|0.17625196367676924|
|         12357126|  44|26099|0.16858883482125753|
|         12786907|  43|26099|0.16475727039350166|
|          7074387|  43|26099|0.16475727039350166|
|          7619515|  43|26099|0.16475727039350166|
|          4138018|  42|26099|0

In [98]:

spark.sql("""

select PARENT_PROGRAM_ID,min(EVENT_DATE),max(EVENT_DATE)  from Program_view where 
EVENT_DATE > '2017-12-31' and 
EVENT_DATE < '2019-01-01'
group by PARENT_PROGRAM_ID order by 3 desc
""")
.show()

+-----------------+--------------------+--------------------+
|PARENT_PROGRAM_ID|     min(EVENT_DATE)|     max(EVENT_DATE)|
+-----------------+--------------------+--------------------+
|             null|2017-12-31 00:00:...|2018-12-31 23:30:...|
|          6338370|2018-11-12 22:30:...|2018-12-31 22:30:...|
|         11858824|2018-12-13 20:30:...|2018-12-31 21:30:...|
|         12573381|2018-12-25 05:00:...|2018-12-31 21:00:...|
|          3373867|2018-05-23 12:00:...|2018-12-31 21:00:...|
|          4844210|2018-12-31 21:00:...|2018-12-31 21:00:...|
|          4785437|2018-12-31 20:30:...|2018-12-31 20:30:...|
|         14737806|2018-10-01 11:30:...|2018-12-31 20:30:...|
|             2990|2018-12-31 19:00:...|2018-12-31 19:00:...|
|         11505825|2018-12-31 19:00:...|2018-12-31 19:00:...|
|          6465044|2018-06-07 19:00:...|2018-12-31 19:00:...|
|         11201265|2018-12-31 19:00:...|2018-12-31 19:00:...|
|           764578|2018-02-06 09:00:...|2018-12-31 18:00:...|
|       

In [99]:
spark.sql("""
select * from Program_view PG join
(SELECT * from Program_view where EVENT_DATE > '2016-12-31' and PARENT_PROGRAM_ID is not null) PARENT

""")
.show()



org.apache.spark.sql.catalyst.parser.ParseException: 

In [15]:

spark.sql("""
select 
PARENT_PROGRAM_ID,MASTER_TITLE,PROGRAM_ID,EPISODE_NUMBER,EVENT_DATE 
from Program_view 
where 
EVENT_DATE > '2016-12-31' and 
PARENT_PROGRAM_ID is not null 
order by 1,5 desc
""")
.show()

+-----------------+--------------------+----------+--------------+--------------------+
|PARENT_PROGRAM_ID|        MASTER_TITLE|PROGRAM_ID|EPISODE_NUMBER|          EVENT_DATE|
+-----------------+--------------------+----------+--------------+--------------------+
|                0|  Extended Hanoi Duo|  14191204|          null|2017-11-11 08:00:...|
|                0|Shabaka Hutchings...|  14191202|          null|2017-11-11 07:00:...|
|          1000058|   The Little People|    315052|            27|2018-05-07 06:40:...|
|          1000058|   The Little People|    315045|            26|2018-04-30 06:50:...|
|          1000058|   The Little People|    315051|            25|2018-04-30 06:40:...|
|         10004862|    My Knight and Me|  12678289|          null|2017-06-30 08:10:...|
|         10004862|    My Knight and Me|  12276284|          null|2017-05-31 12:31:...|
|         10004862|    My Knight and Me|  12276291|          null|2017-05-30 12:31:...|
|         10004862|    My Knight

In [25]:
val program_data = spark.sql("""
select 
PARENT_PROGRAM_ID,MASTER_TITLE,CATEGORY_ID,count(distinct PROGRAM_ID),min(EVENT_DATE),max(EVENT_DATE)
from Program_view 
where 
EVENT_DATE > '2016-12-31' and 
PARENT_PROGRAM_ID is not null 
group by PARENT_PROGRAM_ID,MASTER_TITLE,CATEGORY_ID
order by 4 desc
""")

program_data.show()

+-----------------+--------------------+-----------+--------------------------+--------------------+--------------------+
|PARENT_PROGRAM_ID|        MASTER_TITLE|CATEGORY_ID|count(DISTINCT PROGRAM_ID)|     min(EVENT_DATE)|     max(EVENT_DATE)|
+-----------------+--------------------+-----------+--------------------------+--------------------+--------------------+
|          1055967|         Family Feud|          5|                       205|2017-05-04 04:00:...|2019-02-26 04:30:...|
|          8871523|Los jóvenes titan...|          3|                       134|2017-04-21 20:30:...|2019-02-04 17:45:...|
|          4135805|      Ridiculousness|          5|                       113|2017-01-27 20:00:...|2019-03-20 20:30:...|
|          1471955|    Hora de aventura|          3|                       105|2017-08-04 09:00:...|2019-03-12 00:24:...|
|          6769296|     Teen Titans Go!|          3|                        99|2017-01-05 15:30:...|2019-03-20 14:00:...|
|          4137967|El in

program_data: org.apache.spark.sql.DataFrame = [PARENT_PROGRAM_ID: string, MASTER_TITLE: string ... 4 more fields]


In [23]:
libraryDependencies += "org.vegas-viz" %% "vegas" % {vegas-version}

import vegas._
import vegas.render.WindowRenderer._


<console>: 24: error: not found: value libraryDependencies

In [ ]:
sc.stop()

In [31]:
sc.getConf.toDebugString

res24: String =
spark.app.id=local-1553184329763
spark.app.name=spylon-kernel
spark.driver.host=76f1ec6eb8a7
spark.driver.port=39307
spark.executor.id=driver
spark.master=local[*]
spark.rdd.compress=True
spark.repl.class.outputDir=/tmp/tmpfum57hck
spark.repl.class.uri=spark://76f1ec6eb8a7:39307/classes
spark.serializer.objectStreamReset=100
spark.submit.deployMode=client
spark.ui.showConsoleProgress=true
